In [1]:
pickle_file = 'latest_results.pkl'

In [2]:
import numpy as np
import pandas as pd
import time
import pickle
import matplotlib.pyplot as plt

pd.options.display.max_rows = 2000
pd.options.display.max_columns = 1000
pd.options.display.max_colwidth = 255

In [3]:
with open('./pickles/%s' % pickle_file, 'rb') as f:
    results_df = pickle.load(f)    

In [4]:
def choose_largest_coefficient_name (coefficients):
    if coefficients is None:
        return None
    return coefficients.sort_values(by='abs_weight', ascending=False)['X_Column'].iloc[0]

def choose_largest_coefficient_weight (coefficients):    
    if coefficients is None:
        return None
    return coefficients.sort_values(by='abs_weight', ascending=False)['abs_weight'].iloc[0]
    
    
results_df['largest_coefficient_column'] = results_df['coefficients'].apply(choose_largest_coefficient_name)
results_df['largest_coefficient_weight'] = results_df['coefficients'].apply(choose_largest_coefficient_weight)

In [6]:
results_df \
    .drop(columns=['X_columns', 'coefficients'])

P      N    TP     FP     TN    FN    TPR    TNR  \
country        cutoff                                                        
United States  5000    3747  15592   820    360  15232  2927  0.219  0.977   
               5000    3747  15592   736    105  15487  3011  0.196  0.993   
               5000    3747  15592    32     96  15496  3715  0.009  0.994   
               2500    3747  15592   875    399  15193  2872  0.234  0.974   
               2500    3747  15592   555     76  15516  3192  0.148  0.995   
               2500    3747  15592     7     38  15554  3740  0.002  0.998   
               1000    3747  15592   912    395  15197  2835  0.243  0.975   
               1000    3747  15592   477     62  15530  3270  0.127  0.996   
               1000    3747  15592   516    814  14778  3231  0.138  0.948   
               500     3747  15592   921    418  15174  2826  0.246  0.973   
               500     3747  15592   491     67  15525  3256  0.131  0.996   
               500     3747  15592  3638  12029   3563   109  0.971  0.229   
               250     3747  15592   931    422  15170  2816  0.248  0.973   
               250     3747  15592   521     62  15530  3226  0.139  0.996   
               250     3747  15592   108    145  15447  3639  0.029  0.991   
               100     3747  15592   930    421  15171  2817  0.248  0.973   
               100     3747  15592   489     60  15532  3258  0.131  0.996   
               100     3747  15592  1149   1503  14089  2598  0.307  0.904   
India          5000    2585  16754   408    111  16643  2177  0.158  0.993   
               5000    2585  16754    40      6  16748  2545  0.015  1.000   
               5000    2585  16754   468   2233  14521  2117  0.181  0.867   
               2500    2585  16754   459    129  16625  2126  0.178  0.992   
               2500    2585  16754    21      3  16751  2564  0.008  1.000   
               2500    2585  16754     1      5  16749  2584  0.000  1.000   
               1000    2585  16754   492    133  16621  2093  0.190  0.992   
               1000    2585  16754    17      1  16753  2568  0.007  1.000   
               1000    2585  16754     1    218  16536  2584  0.000  0.987   
               500     2585  16754   500    135  16619  2085  0.193  0.992   
               500     2585  16754    17      1  16753  2568  0.007  1.000   
               500     2585  16754   360   1771  14983  2225  0.139  0.894   
               250     2585  16754   503    132  16622  2082  0.195  0.992   
               250     2585  16754    14      1  16753  2571  0.005  1.000   
               250     2585  16754     4     31  16723  2581  0.002  0.998   
               100     2585  16754   504    134  16620  2081  0.195  0.992   
               100     2585  16754    20      2  16752  2565  0.008  1.000   
               100     2585  16754  2017  11830   4924   568  0.780  0.294   
United Kingdom 5000    1138  18201     0      0  18201  1138  0.000  1.000   
               5000    1138  18201     0      0  18201  1138  0.000  1.000   
               5000    1138  18201     0      0  18201  1138  0.000  1.000   
               2500    1138  18201     0      0  18201  1138  0.000  1.000   
               2500    1138  18201     0      0  18201  1138  0.000  1.000   
               2500    1138  18201  1029  14482   3719   109  0.904  0.204   
               1000    1138  18201     0      0  18201  1138  0.000  1.000   
               1000    1138  18201     0      0  18201  1138  0.000  1.000   
               1000    1138  18201     0      0  18201  1138  0.000  1.000   
               500     1138  18201     0      0  18201  1138  0.000  1.000   
               500     1138  18201     0      0  18201  1138  0.000  1.000   
               500     1138  18201    84    309  17892  1054  0.074  0.983   
               250     1138  18201     0      0  18201  1138  0.000  1.000   
               250     1138  18201     0      0 

In [7]:
def generate_country_aggregates (runs_for_country):
    total_sims = runs_for_country.shape[0]
    nopes =  runs_for_country.query('TP == 0 and FP == 0').shape[0]
    proportion_of_nopes = round(nopes / total_sims, 3)
    country_proportion = round(runs_for_country.iloc[0]['P'] / (runs_for_country.iloc[0]['P'] + runs_for_country.iloc[0]['N']),3)

    result = pd.Series({
        'total_sims' : total_sims,
        'country_proportion': country_proportion,
        'nopes' : nopes,
        'proportion_of_nopes' : proportion_of_nopes,
    })
    
    return result
    
country_aggregates = results_df \
    .reset_index() \
    .drop(columns=['coefficients', 'X_columns']) \
    .groupby('country') \
    .apply(generate_country_aggregates)

country_aggregates

,total_sims,country_proportion,nopes,proportion_of_nopes
country,,,,
Australia,18.0,0.019,13.0,0.722
Brazil,18.0,0.027,15.0,0.833
Canada,18.0,0.035,14.0,0.778
France,18.0,0.029,16.0,0.889
Germany,18.0,0.059,13.0,0.722
India,18.0,0.134,0.0,0.000
Netherlands,18.0,0.020,15.0,0.833
Poland,18.0,0.020,15.0,0.833
United Kingdom,18.0,0.059,15.0,0.833


In [ ]:
(country_aggregates[['proportion_of_nopes', 'country_proportion']]).sort_values(by='country_proportion', ascending=False).plot(kind="bar");
plt.title("Top 10 Countries measured by predictive model failure count");


In [9]:
non_useless_classifiers = results_df \
    .query('TP > 0 or FP > 0') \
    .drop(columns=['X_columns', 'coefficients'])

In [18]:
print("model count %s" % results_df.shape[0])
print("non useless classifiers (i.e., P > 0): %s" % results_df.query('TP > 0 or FP > 0').shape[0])

model count 180
non useless classifiers (i.e., P > 0): 64


In [10]:
non_useless_classifiers.sort_values(by='TPR', ascending=False) \
    .rename(columns = {}) \
    [['model_type', 'column_count', 'P', 'N', 'TPR', 'TNR', 'PPV', 'NPV', 'ACC']] \
    .head(1)

,,model_type,column_count,P,N,TPR,TNR,PPV,NPV,ACC
country,cutoff,,,,,,,,,
United States,500,linear_svm,433,3747,15592,0.971,0.229,0.232,0.97,0.372


In [11]:
non_useless_classifiers.sort_values(by='PPV', ascending=False).head(10) \
    .rename(columns = {}) \
    [['model_type', 'column_count', 'P', 'N', 'TPR', 'TNR', 'PPV', 'NPV', 'ACC']] \
    .head(1)

,,model_type,column_count,P,N,TPR,TNR,PPV,NPV,ACC
country,cutoff,,,,,,,,,
India,500,random_forest,433,2585,16754,0.007,1.0,0.944,0.867,0.867


In [12]:
non_useless_classifiers.sort_values(by='ACC', ascending=False).head(10) \
    .rename(columns = {}) \
    [['model_type', 'column_count', 'P', 'N', 'TPR', 'TNR', 'PPV', 'NPV', 'ACC']] \
    .head(1)

,,model_type,column_count,P,N,TPR,TNR,PPV,NPV,ACC
country,cutoff,,,,,,,,,
Australia,100,linear_svm,441,358,18981,0.0,1.0,0.0,0.981,0.981
